In [2]:
"""
This notebook runs two different versions of finding all nearest
neighbors in a randomly generated set of points.

Change the dimensions of the points and size of data set to
see how the runtime is impacted
"""

import random
from sklearn.neighbors import NearestNeighbors
import numpy as np


def generate_points(n_points, dimensions):
    """Generates poin
    
    Args:
        n_points (int): number of points to generate
        dimensions (int): number of dimensions each point lives in
        
    Returns:
        list
    """
    def gen_point(d):
        return [random.random() for i in range(d)]
    
    return [gen_point(dimensions) for i in range(n_points)]

def distance(p1, p2):
    return sum((i - j)**2 for (i, j) in zip(p1, p2))**0.5

In [3]:
# Naive Implementation O(n^2)
def naive_get_nearest_neighbor(all_points, start_point):
    """Find the nearest point the start_point inside of all_points
    
    Args:
        all_points (list of points): all available points for comparison
        start_point (vector): a single point to center the rest of data set around
        
    Return:
        vector, of the closest point
    """
    closest = all_points[0]
    d_closest = distance(closest, start_point)
    for point in all_points[1:]:
        d_candidate = distance(point, start_point)
        if d_candidate < d_closest:
            closest = point
            d_closest = d_candidate
            
    return closest


def naive_get_all_nn(data):
    return [naive_get_nearest_neighbor(data, point) for point in data]

In [3]:
# Hash points to buckets and then look for nearest neighbors
"""
Two known bugs, nearest neighbors are returned in a scrambled order - need an index to
straighten out. The hashing is not loss-less, it has trouble with boundary conditions.
"""
from collections import defaultdict
hash_vectors = generate_points(3, 10)

def compare(v1, v2):
    over_under = sum(1 if i > j else 0 for (i, j) in zip(v1, v2)) - 5
    return 1 if over_under > 0 else -1

def hash_point(point):
    return (compare(point, h) for h in hash_vectors)

def hashed_get_all_nn(data):
    buckets = defaultdict(list)
    for point in data:
        buckets[hash_point(point)].append(point)
        
    # Note this is slightly buggy as returned in in correct order
    return [naive_get_all_nn(points) for points in buckets.values()]

In [4]:
# Built in Nearest Neighbors
def built_in_get_all_nn(data):
    X = np.array(data)
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(X)
    return nbrs.kneighbors(X)[0]

In [16]:
# Make sure they all run - basic baby problem
data = generate_points(10**1, 10)

print('\nNaive')
%time naive_nn = naive_get_all_nn(data)

print('\nHashed')
%time hashed_nn = hashed_get_all_nn(data)

print('\nBuilt In')
%time built_in_nn = built_in_get_all_nn(data)


Naive
CPU times: user 789 µs, sys: 293 µs, total: 1.08 ms
Wall time: 1.12 ms

Hashed
CPU times: user 5.53 ms, sys: 11.3 ms, total: 16.9 ms
Wall time: 27.2 ms

Built In
CPU times: user 3.19 ms, sys: 14.8 ms, total: 18 ms
Wall time: 27.7 ms


In [17]:
# Slightly Larger
data = generate_points(10**2, 10)

print('Naive')
%time naive_nn = naive_get_all_nn(data)

print('\nHashed')
%time hashed_nn = hashed_get_all_nn(data)

print('\nBuilt In')
%time built_in_nn = built_in_get_all_nn(data)

Naive
CPU times: user 53 ms, sys: 3.45 ms, total: 56.5 ms
Wall time: 67.3 ms

Hashed
CPU times: user 1.01 ms, sys: 113 µs, total: 1.12 ms
Wall time: 1.23 ms

Built In
CPU times: user 1.66 ms, sys: 455 µs, total: 2.12 ms
Wall time: 1.8 ms


In [18]:
# Make sure they all work
data = generate_points(10**3, 10)

print('Naive')
%time naive_nn = naive_get_all_nn(data)

Naive
CPU times: user 3.96 s, sys: 59.9 ms, total: 4.02 s
Wall time: 4.56 s


In [19]:
print('Hashed')
%time hashed_nn = hashed_get_all_nn(data)

Hashed
CPU times: user 11 ms, sys: 1.03 ms, total: 12 ms
Wall time: 12.4 ms


In [20]:
print('Built In')
%time built_in_nn = built_in_get_all_nn(data)

Built In
CPU times: user 25.7 ms, sys: 2.85 ms, total: 28.5 ms
Wall time: 29 ms


In [21]:
# Let's try bigger data
data = generate_points(10**4, 10)

In [22]:
print('Naive')
%time naive_nn = naive_get_all_nn(data)

Naive
CPU times: user 9min 20s, sys: 11.4 s, total: 9min 32s
Wall time: 13min 35s


In [13]:
print('Hashed')
%time hashed_nn = hashed_get_all_nn(data)

Hashed
CPU times: user 99.1 ms, sys: 7.24 ms, total: 106 ms
Wall time: 105 ms


In [14]:
print('Built In')
%time built_in_nn = built_in_get_all_nn(data)

Built In
CPU times: user 885 ms, sys: 5.55 ms, total: 891 ms
Wall time: 892 ms
